# Suicide worldwide 1985-2016

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import re

* Este dataset contiene país, año, grupo de edad y generación, población, rate de suicio y otros valores que analizaremos más en detalle.

* Recoge datos entre 1986 y 2016 que veremos si podemos aumentar posteriormente.

* Los países del dataset son mayormente países de Norte América y Sudamérica, Europa, Oceanía, Sudáfrica, Mongolia, Japón...exluyendo Rusia y gran parte de África.

* Se intentará relacionar los datos de este dataset con condiciones externas como guerras, consumo de drogas, el tiempo en esos países, religión, redes sociales...

In [2]:
data = pd.read_csv("data/master.csv",encoding='ISO-8859-1')
data.head()

,ï»¿country,year,sex,age,suicides_no,population,suicides/100k pop,country-year,HDI for year,gdp_for_year ($),gdp_per_capita ($),generation
0,Albania,1987,male,15-24 years,21,312900,6.71,Albania1987,NaN,"2,156,624,900",796,Generation X
1,Albania,1987,male,35-54 years,16,308000,5.19,Albania1987,NaN,"2,156,624,900",796,Silent
2,Albania,1987,female,15-24 years,14,289700,4.83,Albania1987,NaN,"2,156,624,900",796,Generation X
3,Albania,1987,male,75+ years,1,21800,4.59,Albania1987,NaN,"2,156,624,900",796,G.I. Generation
4,Albania,1987,male,25-34 years,9,274300,3.28,Albania1987,NaN,"2,156,624,900",796,Boomers


# Tipo de datos

In [3]:
data.dtypes

ï»¿country             object
year                    int64
sex                    object
age                    object
suicides_no             int64
population              int64
suicides/100k pop     float64
country-year           object
HDI for year          float64
 gdp_for_year ($)      object
gdp_per_capita ($)      int64
generation             object
dtype: object

# Missing values

In [4]:
null_cols = data.isnull().sum()
null_cols[null_cols > 0]

HDI for year    19456
dtype: int64

Observamos que hay una única columna con valores nulos. Esta columna es HDI for year (Human Development Index), que tiene 19.456 de 27.820 valores nulos. Ya que solo existe este dato en algunos países, no tiene sentido mantenerlo porque no se va a poder comparar con el resto.

In [5]:
data = data.drop('HDI for year', axis=1)

También voy a dropear la columna country-year ya que esta info está separada en country y year en otras dos columnas. 

In [6]:
data = data.drop('country-year', axis=1)

# Duplicates

In [7]:
before = len(data)
data = data.drop_duplicates()
after = len(data)
print('Número de duplicados dropped: ', str(before - after))

Número de duplicados dropped:  0


No hay duplicados.

In [8]:
data

,ï»¿country,year,sex,age,suicides_no,population,suicides/100k pop,gdp_for_year ($),gdp_per_capita ($),generation
0,Albania,1987,male,15-24 years,21,312900,6.71,"2,156,624,900",796,Generation X
1,Albania,1987,male,35-54 years,16,308000,5.19,"2,156,624,900",796,Silent
2,Albania,1987,female,15-24 years,14,289700,4.83,"2,156,624,900",796,Generation X
3,Albania,1987,male,75+ years,1,21800,4.59,"2,156,624,900",796,G.I. Generation
4,Albania,1987,male,25-34 years,9,274300,3.28,"2,156,624,900",796,Boomers
...,...,...,...,...,...,...,...,...,...,...
27815,Uzbekistan,2014,female,35-54 years,107,3620833,2.96,"63,067,077,179",2309,Generation X
27816,Uzbekistan,2014,female,75+ years,9,348465,2.58,"63,067,077,179",2309,Silent
27817,Uzbekistan,2014,male,5-14 years,60,2762158,2.17,"63,067,077,179",2309,Generation Z
27818,Uzbekistan,2014,female,5-14 years,44,2631600,1.67,"63,067,077,179",2309,Generation Z


# Rename columns

In [9]:
data.rename(columns={'ï»¿country'        :'country',
                     'age'               : 'age_range',
                     'suicides_no'       : 'suicide_number',
                     'suicides/100k pop' : 'suicides/100k',
                     ' gdp_for_year ($) ': 'gdp_for_year',
                     'gdp_per_capita ($)': 'gdp_per_capita'                    
                    }, inplace=True)

In [10]:
data.head()

,country,year,sex,age_range,suicide_number,population,suicides/100k,gdp_for_year,gdp_per_capita,generation
0,Albania,1987,male,15-24 years,21,312900,6.71,"2,156,624,900",796,Generation X
1,Albania,1987,male,35-54 years,16,308000,5.19,"2,156,624,900",796,Silent
2,Albania,1987,female,15-24 years,14,289700,4.83,"2,156,624,900",796,Generation X
3,Albania,1987,male,75+ years,1,21800,4.59,"2,156,624,900",796,G.I. Generation
4,Albania,1987,male,25-34 years,9,274300,3.28,"2,156,624,900",796,Boomers


# Columns

Breve explicación de los datos de las columnas:
- **country:** país.
- **year:** año.
- **sex:** solo toma dos valores, male o female.
- **age_range:** solo toma 5 valores:
  - 15-24 years
  - 25-34 years     
  - 35-54 years 
  - 55-74 years 
  - 75+ years      

- **suicide_number:** numero de suicidios en ese país y año en ese rango de edad.
- **population:** población del país.
- **suicides/100k:** suicidios por cada 100mil habitantes.
- **gdp_for_year:** pib anual.
- **gdp_per_capita:** pib per capita.
- **generation:** generación a la que pertenece la persona. Toma 5 valores.
  - Generation X       
  - Silent             
  - Millenials         
  - Boomers            
  - G.I. Generation   

# Export

In [11]:
data.to_csv('clean.csv')